# Fortran-TF-lib

Source: https://github.com/Cambridge-ICCS/fortran-tf-lib

## TensorFlow C API

https://www.tensorflow.org/install/lang_c

- baixa e instala :

```
FILENAME=libtensorflow-cpu-linux-x86_64.tar.gz
wget -q --no-check-certificate https://storage.googleapis.com/tensorflow/versions/2.18.1/${FILENAME}
sudo tar -C /usr/local -xzf ${FILENAME}
```

- configura o linker :

```
sudo ldconfig /usr/local/lib
```

- testa:

In [1]:
%%writefile hello_tf.c
#include <stdio.h>
#include <tensorflow/c/c_api.h>

int main() {
  printf("Hello from TensorFlow C library version %s\n", TF_Version());
  return 0;
}

Writing hello_ff.c


In [5]:
! gcc hello_tf.c -ltensorflow -o hello_tf

In [7]:
! ./hello_tf

Hello from TensorFlow C library version 2.18.1


## Library installation

```
git clone git@github.com:Cambridge-ICCS/fortran-tf-lib.git
cd fortran-tf-lib/fortran-tf-lib/
mkdir build
cd build
cmake .. -DCMAKE_BUILD_TYPE=Release -DTENSORFLOW_LOCATION=/usr/local/lib
make
sudo make install
```

/usr/local/include/ - contains mod files

/usr/local/lib64/ - contains cmake directory and .so files

```
-- Install configuration: "Release"
-- Installing: /usr/local/lib/libfortran-tf.so.0.1
-- Set runtime path of "/usr/local/lib/libfortran-tf.so.0.1" to "$ORIGIN/../lib:/usr/local/lib"
-- Installing: /usr/local/lib/libfortran-tf.so
-- Installing: /usr/local/lib/cmake/FortranTensorFlowConfig.cmake
-- Installing: /usr/local/lib/cmake/FortranTensorFlowConfig-release.cmake
-- Installing: /usr/local/include/fortran-tf/tf_interface.mod
-- Installing: /usr/local/include/fortran-tf/tf_types.mod
```


## Usage

1. Save a TensorFlow model in the Keras SavedModel format.
2. Write Fortran using the fortran-tf-lib bindings to use the model from within Fortran.
3. Build and compile the code, linking against fortran-tf-lib.

### Saving the model

The trained model needs to be exported. This can be done from within your code using the model.save functionality from within python. Note that the TensorFlow C API currently (version 2.13) only supports the Keras "v2" format so you must specify format='tf':

        import tensorflow as tf
        # construct model (e.g. model=tf.keras.Model(inputs, outputs))
        # or load one (e.g. model=tf.keras.models.load_model('/path/to/model'))
        model.save("my_model", format='tf')

### Using the model from Fortran

To use the trained TensorFlow model from within Fortran we need to import the TF_Interface module and use the binding routines to load the model, construct the tensors, and run inference.

```fortran
program test_program
use TF_Types
use TF_Interface
use iso_c_binding
implicit none

type(TF_Session) :: session
type(TF_SessionOptions) :: sessionoptions
type(TF_Graph) :: graph
type(TF_Status) :: stat
type(TF_Output), dimension(1) :: input_tfoutput, output_tfoutput
character(100) :: vers
character(100), dimension(1) :: tags
type(TF_Tensor), dimension(1) :: input_tensors, output_tensors, test_tensor
type(TF_Operation), dimension(1) :: target_opers

real, dimension(32), target :: raw_data
real, dimension(:), pointer :: output_data_ptr
integer(kind=c_int64_t), dimension(2) :: input_dims
integer(kind=c_int64_t), dimension(2) :: output_dims
type(c_ptr) :: raw_data_ptr
type(c_ptr) :: output_c_data_ptr

raw_data = (/ &
        0.71332126, 0.81275973, 0.66596436, 0.79570779, 0.83973302, 0.76604397, &
        0.84371391, 0.92582056, 0.32038017, 0.0732005, 0.80589203, 0.75226581, &
        0.81602784, 0.59698078, 0.32991729, 0.43125108, 0.4368422, 0.88550326, &
        0.7131253, 0.14951148, 0.22084413, 0.70801317, 0.69433906, 0.62496564, &
        0.50744999, 0.94047845, 0.18191579, 0.2599102, 0.53161889, 0.57402205, &
        0.50751284, 0.65207096 &
        /)


input_dims = (/ 1, 32 /)
output_dims = (/ 1, 1 /)
tags(1) = 'serve'

! Print TensorFlow library version
call TF_Version(vers)
write(*,*)'Tensorflow version', vers

sessionoptions = TF_NewSessionOptions()
graph = TF_NewGraph()
stat = TF_NewStatus()

! Load session (also populates graph)
session = TF_LoadSessionFromSavedModel(sessionoptions, '/path/to/model', tags, 1, &
    graph, stat)

if (TF_GetCode( stat ) .ne. TF_OK) then
    call TF_Message( stat, vers )
    write(*,*)'woops', TF_GetCode( stat ), vers
    call abort
endif

call TF_DeleteSessionOptions(sessionoptions)

input_tfoutput(1)%oper = TF_GraphOperationByName( graph, "serving_default_input_1" )
input_tfoutput(1)%index = 0
if (.not.c_associated(input_tfoutput(1)%oper%p)) then
    write(*,*)'input not associated'
    stop
endif

output_tfoutput(1)%oper = TF_GraphOperationByName( graph, "StatefulPartitionedCall" )
output_tfoutput(1)%index = 0
if (.not.c_associated(output_tfoutput(1)%oper%p)) then
    write(*,*)'output not associated'
    stop
endif

! Bind the input tensor
raw_data_ptr = c_loc(raw_data)
input_tensors(1) = TF_NewTensor( TF_FLOAT, input_dims, 2, raw_data_ptr, int(128, kind=c_size_t) )

! Run inference
call TF_SessionRun( session, input_tfoutput, input_tensors, 1, output_tfoutput, output_tensors, 1, &
    target_opers, 0, stat )
if (TF_GetCode( stat ) .ne. TF_OK) then
    call TF_Message( stat, vers )
    write(*,*) TF_GetCode( stat ), vers
    call abort
endif

! Bind output tensor
call c_f_pointer( TF_TensorData( output_tensors(1)), output_data_ptr, shape(output_data_ptr) )
write(*,*)'output data', output_data_ptr(1)

if ((output_data_ptr(1) - -0.479371) .gt. 1e-6) then
    write(*,*)'Output does not match, FAILED!'
else
    write(*,*)'Output is correct, SUCCESS!'
endif


! Clean up
call TF_DeleteTensor( input_tensors(1) )
call TF_DeleteTensor( output_tensors(1) )
call TF_DeleteGraph( graph )
call TF_DeleteSession( session, stat )
call TF_DeleteStatus( stat )

end program test_program
```

### Generating code with process_model

The example code above illustrates a problem with the TensorFlow C API that our Fortran wrapper cannot fix. To load a model, the library requires that the caller knows certain rather opaque model parameters beforehand. Often, the values in the example above will work for the tags parameter to TF_LoadSessionFromSavedModel. However, the values needed for TF_GraphOperationByName (in this case serving_default_input_1, etc) are more likely to be different.

To address this, we provide a Python script, process_model that will read a Keras SavedModel and output a simple Fortran module intended to provide a base for the user to start from. The appropriate values will be read from the model and hard-coded into the Fortran code.

E.g.

        process_model -o fortran_code.f90  my_model

## Build the code

The code now needs to be compiled and linked against our installed library.


### CMake

If our project were using cmake we would need the following in the CMakeLists.txt file to find the the tf-lib installation and link it to the executable.

This can be done by adding the following to the CMakeLists.txt file:

        find_package(FortranTensorFlow)
        target_link_libraries( <executable> PRIVATE FortranTensorFlow::fortran-tf )
        message(STATUS "Building with Fortran TensorFlow coupling")

Running cmake:

        mkdir build
        cd build
        cmake .. -DCMAKE_PREFIX_PATH=</path/to/fortran-tf-libs/lib64/cmake> -DCMAKE_BUILD_TYPE=Release
        cmake --build .

When running the generated code you may also need to add the location of the .so files to your LD_LIBRARY_PATH unless installing in a default location:

        export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path/to/fortran-tf-libs>/lib64
